In [1]:
import pandas as pd

In [ ]:
def get_intra_worker_paraphrased_examples(row):
    intra_worker_paraphrased_examples = []
    
    for worker_num, worker_paraphrases in enumerate(row.paraphrases): # 3 workers
        for h1_id, h1 in enumerate(worker_paraphrases['hyp1_paraphrases']): # each worker had 3 hyp1s and 3 hyp2s
            for h2_id, h2 in enumerate(worker_paraphrases['hyp2_paraphrases']):
                intra_worker_paraphrased_examples.append({
                    'obs1': row.obs1, 
                    'obs2': row.obs2,
                    'label': row.label,
                    'hyp1': h1,
                    'hyp2': h2,
                    'paraphrase_id': f'{row.example_id}.{worker_num}.{h1_id}.{h2_id}',
                    'original_hypotheses': {
                        'hyp1': row.hyp1,
                        'hyp2': row.hyp2
                    }
                })
    return intra_worker_paraphrased_examples

In [3]:
def get_zipped_intra_worker_paraphrased_examples(row):
    intra_worker_paraphrased_examples = []
    
    for worker_num, worker_paraphrases in enumerate(row.paraphrases): # 3 workers
        for id_, (h1, h2) in enumerate(list(zip(worker_paraphrases['hyp1_paraphrases'], worker_paraphrases['hyp2_paraphrases']))):
            intra_worker_paraphrased_examples.append({
                'obs1': row.obs1, 
                'obs2': row.obs2,
                'label': row.label,
                'hyp1': h1,
                'hyp2': h2,
                'paraphrase_id': f'{row.example_id}.{worker_num}.{id_}',
                'original_hypotheses': {
                    'hyp1': row.hyp1,
                    'hyp2': row.hyp2
                }
            })
    return intra_worker_paraphrased_examples

In [4]:
pilot_tasks = pd.read_csv('../annotated-data/abductive/paraphrased_pilot.csv').drop(columns=[
    'story_id', 'origin_split', 'example_data', 'processed_assignments'
])

pilot_tasks['paraphrases'] = pilot_tasks['paraphrases'].map(eval)
pilot_tasks['intra_worker_paraphrases'] = pilot_tasks.apply(get_zipped_intra_worker_paraphrased_examples, axis=1)

In [5]:
intra_worker_paraphrases = pd.DataFrame([example for paraphrase_group in pilot_tasks.intra_worker_paraphrases for example in paraphrase_group])

In [6]:
len(intra_worker_paraphrases)

1035

In [7]:
intra_worker_paraphrases.to_csv('zipped_intra_worker_paraphrases_pilot.csv', index=False)